In [78]:
import pandas as pd
import gensim
from gensim import corpora
from gensim.models import TfidfModel, Phrases
import nltk
from nltk.corpus import reuters
import spacy

nltk.download('reuters')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\rachi\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [88]:
nlp = spacy.load("en_core_web_sm")
corpus = reuters.sents(categories=["cpu"])

# Convert the corpus to a list of sentences
example_text = [" ".join(sent) for sent in corpus]

texts = []
for document in example_text:
    doc = nlp(document)
    text = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    texts.append(text)

dictionary = corpora.Dictionary(texts)
corpus_bow = [dictionary.doc2bow(text) for text in texts]

# Create a TF-IDF model
tfidf_model = TfidfModel(corpus_bow)
corpus_tfidf = tfidf_model[corpus_bow]

bigram = Phrases(texts, min_count=1, threshold=20)
texts_with_bigrams = [bigram[line] for line in texts]

trigram = Phrases(texts_with_bigrams, min_count=1, threshold=1)
texts_with_trigrams = [trigram[line] for line in texts_with_bigrams]

print("Bigrams:")
for text in texts_with_bigrams:
        for word in text:
               if word.count("_") == 1:
                print(word)


Bigrams:
u_s
INDUSTRIAL_CAPACITY
USE_RATE
81_2
PCT_SEPTEMBER
UNCHANGED_AUGUST
u_s
INDUSTRIAL_CAPACITY
USE_RATE
81_2
PCT_SEPTEMBER
UNCHANGED_AUGUST
u_s
INDUSTRIAL_CAPACITY
USE_RATE
PCT_FEB
PCT_JAN
u_s
INDUSTRIAL_CAPACITY
USE_RATE
PCT_FEB
PCT_JAN
u_s
CAPACITY_USE
primary_metal
Fed_say
February_1986
81_2
primary_metal
February_1986
Fed_say
producer_industrial
February_1986
Fed_say
producer_industrial


In [89]:
print("\nTrigrams:")
for text in texts_with_trigrams:
        for word in text:
               if word.count("_") == 2:
                print(word)


Trigrams:
USE_RATE_ROSE
PCT_FEB_79
6_PCT_JAN
USE_RATE_ROSE
PCT_FEB_79
6_PCT_JAN
February_1986_rate
producer_industrial_material
February_1986_rate
producer_industrial_material


In [90]:
df = pd.DataFrame({"text": texts }) # create a dataframe with the results
# df["Tfidf"] = df["Tfidf"].apply(lambda x: sorted(x, key=lambda tup: tup[1], reverse=True)) # sort the tuples by the tfidf value
# df["Tfidf"] = df["Tfidf"].apply(lambda x: [dictionary[tup[0]] for tup in x]) # convert the ids to words
# df["Tfidf"] = df["Tfidf"].apply(lambda x: x[:3]) # keep only the top 3 words
# # Convert the lists to strings and remove brackets
# df["Tfidf"] = df["Tfidf"].apply(lambda x: ', '.join(x))
df["text"] = df["text"].apply(lambda x: ', '.join(x))

df["bigrams"] = texts_with_bigrams[:] # add the bigrams to the dataframe


df["bigrams"] = df["bigrams"].apply(lambda x: ', '.join(x[:3])) # convert the lists to strings and remove brackets
df["trigrams"] = texts_with_trigrams # add the bigrams and trigrams to the dataframe
df["trigrams"] = df["trigrams"].apply(lambda x: ', '.join(x[:2])) # convert the lists to strings and remove brackets
df.head(10)


,text,bigrams,trigrams
0,"u, s, INDUSTRIAL, CAPACITY, USE, RATE, 81, 2, ...","u_s, INDUSTRIAL_CAPACITY, USE_RATE","u_s_INDUSTRIAL_CAPACITY, USE_RATE_81_2"
1,"u, s, INDUSTRIAL, CAPACITY, USE, RATE, 81, 2, ...","u_s, INDUSTRIAL_CAPACITY, USE_RATE","u_s_INDUSTRIAL_CAPACITY, USE_RATE_81_2"
2,"CANADA, MANUFACTURING, UTILIZATION, rate, rise...","CANADA, MANUFACTURING, UTILIZATION","CANADA, MANUFACTURING"
3,"change, small, mark, quarter, quarter, 1985, u...","change, small, mark","change, small"
4,"increase, residential, construction, lead, str...","increase, residential, construction","increase, residential"
5,"u, s, INDUSTRIAL, CAPACITY, USE, RATE, ROSE, 7...","u_s, INDUSTRIAL_CAPACITY, USE_RATE","u_s_INDUSTRIAL_CAPACITY, USE_RATE_ROSE"
6,"u, s, INDUSTRIAL, CAPACITY, USE, RATE, ROSE, 7...","u_s, INDUSTRIAL_CAPACITY, USE_RATE","u_s_INDUSTRIAL_CAPACITY, USE_RATE_ROSE"
7,"u, s, CAPACITY, USE, RATE, 79, 8, PCT, FEBRUAR...","u_s, CAPACITY_USE, RATE","u_s, CAPACITY_USE"
8,"Fed, previously, say, rate, 79, 7, pct, Januar...","Fed, previously, say","Fed, previously"
9,"surge, automobile, assembly, February, gain, p...","surge, automobile, assembly","surge, automobile"


In [82]:
categories = reuters.categories() # get all the categories
categories = pd.DataFrame({"category": categories}) # create a dataframe with the categories
categories[0:10]


,category
0,acq
1,alum
2,barley
3,bop
4,carcass
5,castor-oil
6,cocoa
7,coconut
8,coconut-oil
9,coffee
